<a href="https://colab.research.google.com/github/mrdo19111911/3dbinpacking/blob/1.x/3D_Loading_Divide_space_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numba
from numba import njit

In [2]:
import random as rd
import time



In [3]:
DATA_SIZE=5
ROTATE_FIX=   np.array([
        [0,1,2],
        [0,2,1],
        [1,0,2],
        [1,2,0],
        [2,1,0],
        [2,0,1]], np.int_)

In [4]:
class Bin:
    def __init__(self, size):
        self.size =size
    def print(self):
        print("size", self.size)
class Item:
    def __init__(self, size):
        self.size= size
    def print(self):
        print("data", self.size)

In [5]:
@njit
def compute_rotate_item_size(size, k):
    

    new =  ROTATE_FIX[k]
    # aa =  [size[new[0]], size[new[1]], size[new[2]]]
    new_size =  np.concatenate((np.array([size[new[0]], size[new[1]], size[new[2]]], np.int_) , size[3:]   ))
    
    return new_size

In [6]:
@njit
def compute_number_of_fit(bin_size,item_size): #compute number of fit item
    M_l=math.floor(bin_size[0]/item_size[0])
    M_w=math.floor(bin_size[1]/item_size[1])
    M_h=math.floor(bin_size[2]/item_size[2])
    result = np.empty((4), np.int_)
    result[0]= M_h*M_l*M_w
    result[1]= M_l
    result[2]= M_w
    result[3]= M_h
    return result

@njit

def find_volume_coef(bin_size,item_size ):
    a=math.floor(bin_size[0]/item_size[0])* math.floor(bin_size[1]/item_size[1])* math.floor(bin_size[2]/item_size[2])
    if a==0:
        return 1000000000
    else:
        return ((bin_size[0]/item_size[0])*(bin_size[1]/item_size[1])*(bin_size[2]/item_size[2]))/a

In [7]:
@njit 
def find_number_of_rotate_fit(bin_size,item_size):
    best_num_item = -3
    best_k=-1
    for k in range(6):
        num_item = compute_number_of_fit( bin_size, compute_rotate_item_size(item_size, k))[0] 
        if num_item>best_num_item:
            best_num_item =  num_item
            best_k=k
    result =  np.empty((2), np.int_)
    result[0] =  best_num_item
    result[1] = best_k
    
    return result

In [8]:
@njit
def decompose_bin(bin_size, item_size):#decompose box
    box= np.empty((6, 6), np.int_)
    temp=compute_number_of_fit(bin_size,item_size)
    # print(temp)
    M_lwh, M_l, M_w, M_h= temp[0], temp[1], temp[2], temp[3]
    if M_lwh==0:
        return box
    bin_l = bin_size[0]
    bin_w= bin_size[1]
    bin_h =bin_size[2]
    item_l, item_w, item_h, item_num = item_size
    # print("1")
    temp_num = item_num
    if temp_num>M_lwh:
        temp_num= M_lwh
    OQ= bin_l
    N_l = math.ceil(temp_num/(M_w* M_h)) #8
    num_item_front=temp_num- (N_l-1)*M_w* M_h #20-2*4*2
    OF = N_l*item_l
    # print(N_l)
    # print('need to compute')
    HI = M_w*item_w
    IJ = bin_w-HI
    JP =  math.floor(temp_num/(M_w* M_h))*item_l
    num_colum_face= math.ceil(num_item_front/M_h)
    FA =num_colum_face*item_w
    AB = (M_h-(num_colum_face*M_h-num_item_front))*item_h
    num_box_AB =  num_item_front%(M_h)
    FC= math.floor(num_item_front/M_h )*item_w
    CA = FA-FC

    # print(num_box_AB)
    # print("OF", OF,"FC", FC,"CA","FA",FA, CA, "AB", AB)
    FD= M_h*item_h
    box[0] = [bin_l-OF, bin_w, bin_h, OF, 0, 0]
    box[1] =[ item_l, bin_w-FA, bin_h, OF- item_l, FA, 0]
    box[2] =[ item_l, FA-FC, bin_h- AB, OF-item_l, FC,0]
    box[3] =[ item_l, FC, bin_h- FD, OF- item_l, 0, FD]


    box[4] =[ OF- item_l, bin_w-HI, bin_h, 0,HI, 0]
    box[5]= [OF- item_l, HI, bin_h-FD, 0, 0, FD ]


    return box




In [9]:

@njit
def decompose_to_two_bin(bin_size, item_size):
    for k in range( math.floor(bin_size[1]/item_size[1]/2)+1):
        
        bin_size_1= np.copy(bin_size)
        bin_size_2= np.copy(bin_size)
        # k=1
        bin_size_1[1] = bin_size_1[1]-k* item_size[1]
        bin_size_2[1] = k* item_size[1]
        a,_=find_number_of_rotate_fit(bin_size_1,item_size)
        b,_=find_number_of_rotate_fit(bin_size_2,item_size)

        print(a+b)


In [10]:
time1= time.time()
bin_list_obj = []
bin_list_obj.append(Bin(np.array([10000, 2500,2500,0,0,0], np.int_)))
# bin_list_obj.append(Bin(np.array([10000, 200,500,4], np.int_)))
bin_list =[item.size for item in bin_list_obj]


item_list_obj = []
# item_list_obj.append(Item(np.array([350, 500, 600, 10000], np.int_)))
# item_list_obj.append(Item(np.array([350, 250, 200,11000], np.int_)))
for i in range(1000):
    x=rd.randint(100, 600)
    y=rd.randint(100, 600)
    z=rd.randint(100, 600)
    t=rd.randint(100, 6000)
    item_list_obj.append(Item(np.array([x, y, z,t], np.int_)))
item_list=[item.size for item in item_list_obj]
min_item_dim  =  min([min(item[0], item[1], item[2])  for item in item_list] )
item_list.sort(key=lambda x:-max(x[0], x[1], x[2]))
for i in range(len(item_list)):
    # print("________________")

    # print("number of item before pack", sum([item[3]for item in item_list]))
    item =item_list[i]
    # print("prepare to pack", item)
    temp_bin_list =  []
    if len(bin_list)==0:
        break
    bin_list.sort(key =  lambda x: min(x[0], x[1], x[2]))


    # print("bin_list", bin_list)


    for b in range(len(bin_list)):
        # print(item_list[i][3])
        # print("\tpack this bin", bin_list[b])
        if min(bin_list[b][0], bin_list[b][1], bin_list[b][2]) < min_item_dim :
            continue
        if min(bin_list[b][0], bin_list[b][1], bin_list[b][2]) < min( [item[0], item[1], item[2] ]) :
            temp_bin_list.append(bin_list[b])
            continue


        number_of_fit = compute_number_of_fit(bin_list[b],item)[0]
        if number_of_fit==0:
            temp_bin_list.append(bin_list[b])
            continue
        
        else:
            box = decompose_bin(bin_list[b],item) 
            # number_of_rotate_fit(bin_size,item_size)


            for k in range(len(box)):
                if min(box[k][0], box[k][1], box[k][2]) < min_item_dim :
                    continue
                
                temp_bin_list.append(box[k])

            # print("\tafter adding boxes", temp_bin_list)

        
        if item_list[i][3] >= number_of_fit:
            item_list[i][3] = item_list[i][3]- number_of_fit
        else:
            item_list[i][3] =0
        # print("\tnumber of item left", sum([item[3]for item in item_list]))

        
        
    bin_list= temp_bin_list
    # print("len(bin_list)",len(bin_list))
    if i >= len(item_list):
        break
# print("final")
# print("number of item left", sum([item[3]for item in item_list]))
# print(temp_bin_list)    # print([item.q for item in item_list] )
print(time.time()-time1)
temp_bin_list

0.8744857311248779


[array([ 247,  142,  100,    0, 2072,    0]),
 array([ 109,  100,  172,  109,    0, 2328]),
 array([ 109,  100,  172,    0,    0, 2328])]

t_obj.append(Item(np.array([350, 500, 600, 10000], np.int_)))
item_list_obj.append(Item(np.array([350, 250, 200,11000], np.int_)))